In [10]:
from combined_player import player_minutes_value
from html_scraper import db
from mongo_to_db import create_master_df, create_master_player_min_df

In [2]:
import pprint
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.max_columns', 50)


In [4]:
xg_df = pd.read_csv('xgboost_table.csv')

In [21]:
len(xg_df)

470

In [42]:
xg_df.head()

,Unnamed: 0,player_id,player_name,club,birthday,age,foot,squad_num,total_xG,total_xA,total_xG+xA,goals,xG+xA/90,transfer_value(USD),total_minutes_played
0,0,24256.0,Emmanuel Gigliotti,Squad Club Atlético Independiente,"May 20, 1987 (31)",31,right,9.0,6.24,2.84,9.08,12.0,0.84,3.43,977.97
1,1,488.0,Lisandro López,Squad Racing Club,"Mar 2, 1983 (35)",35,right,15.0,5.26,1.27,6.53,9.0,0.63,2.01,937.47
2,2,16081.0,Luis Rodríguez,Squad Club Atlético Tucumán,"Jan 1, 1985 (33)",33,right,7.0,5.01,1.55,6.56,7.0,0.71,1.71,827.90
3,3,101.0,Darío Cvitanich,Squad Club Atlético Banfield,"May 16, 1984 (34)",34,right,20.0,4.73,1.17,5.90,5.0,0.55,1.71,969.08
4,4,93853.0,Franco Soldano,Squad Club Atlético Unión (Santa Fe),"Sep 14, 1994 (24)",24,right,27.0,4.25,2.27,6.52,2.0,0.55,5.71,1060.35


In [23]:
xg_players = xg_df['player_id'].unique()

# Get positions from player data

In [39]:
games = db.games.find()
players = db.players.find()

In [40]:
final_df = player_minutes_value(games, players)

In [54]:
final_df[final_df['position_id'] == 5]

,player_id,player_name,club,birthday,age,foot,position_id,squad_num,height,total_minutes_played,transfer_value(sterlings),transfer_value(USD)
35,41500.0,Omar De Felippe,Squad Club Atlético Newell's Old Boys,"Feb 17, 1998 (20)",20,left,5,NaN,"1,65 m",1170.0,£270k,0.34
36,41500.0,Omar De Felippe,Squad Club Atlético Newell's Old Boys,"Mar 19, 1998 (20)",20,-,5,NaN,m,1170.0,-,NaN
158,190907.0,Juan Pablo Pumpido,Squad Club Atlético Patronato de la Juventud C...,"Oct 4, 1998 (20)",20,-,5,NaN,m,450.0,-,NaN
159,101207.0,None,Squad Club Atlético Patronato de la Juventud C...,"Oct 4, 1998 (20)",20,-,5,NaN,m,90.0,-,NaN
160,1950.0,Mario Sciacqua,Squad Club Atlético Patronato de la Juventud C...,"Oct 4, 1998 (20)",20,-,5,NaN,m,630.0,-,NaN
248,47815.0,Juan Pablo Vojvoda,Squad Club Atlético Talleres,"Feb 8, 1998 (20)",20,-,5,NaN,m,1170.0,-,NaN
442,162734.0,Ariel Holan,Squad Club Atlético Independiente,"Apr 11, 1993 (25)",25,right,5,NaN,"1,86 m",1080.0,£180k,0.23


In [71]:
final_df['position_id'] = final_df['position_id'].replace('goalie', 'Goalie')
final_df['position_id'] = final_df['position_id'].replace(2, 'Defender')
final_df['position_id'] = final_df['position_id'].replace(3, 'Midfielder')
final_df['position_id'] = final_df['position_id'].replace(4, 'Forward')
final_df['position_id'] = final_df['position_id'].replace(5, 'Defender')

In [72]:
final_df.head()

,player_id,player_name,club,birthday,age,foot,position_id,squad_num,height,total_minutes_played,transfer_value(sterlings),transfer_value(USD)
0,19132.0,Alexander Domínguez,Squad Club Atlético Vélez Sarsfield,"Jun 5, 1987 (31)",31,right,Goalie,22.0,"1,95 m",1080.000000,£1.58m,2.01
1,27987.0,Lucas Hoyos,Squad Club Atlético Vélez Sarsfield,"Apr 29, 1989 (29)",29,right,Goalie,12.0,"1,83 m",90.000000,£135k,0.17
2,30896.0,Gastón Díaz,Squad Club Atlético Vélez Sarsfield,"Mar 13, 1988 (30)",30,right,Defender,24.0,"1,75 m",896.483333,£450k,0.57
3,94225.0,Joaquín Laso,Squad Club Atlético Vélez Sarsfield,"Jul 4, 1990 (28)",28,right,Defender,6.0,"1,85 m",990.000000,£270k,0.34
4,99779.0,Luis Abram,Squad Club Atlético Vélez Sarsfield,"Feb 27, 1996 (22)",22,left,Defender,29.0,"1,80 m",1080.000000,£405k,0.51


In [73]:
players_position = final_df.loc[:, ['player_name', 'position_id']].copy()

In [74]:
df_1 = pd.merge(xg_df, players_position, on=['player_name'])

In [75]:
df_1.columns

Index(['Unnamed: 0', 'player_id', 'player_name', 'club', 'birthday', 'age',
       'foot', 'squad_num', 'total_xG', 'total_xA', 'total_xG+xA', 'goals',
       'xG+xA/90', 'transfer_value(USD)', 'total_minutes_played',
       'position_id'],
      dtype='object')

In [76]:
columns = ['Unnamed: 0', 'player_id', 'player_name', 'club', 'birthday', 'age',
       'foot', 'position_id', 'squad_num', 'total_xG', 'total_xA', 'total_xG+xA', 'goals',
       'xG+xA/90', 'transfer_value(USD)', 'total_minutes_played']

In [77]:
df_2 = df_1[columns]

In [78]:
df_2.to_csv('xgboost_table1.csv')

In [79]:
test_df = pd.read_csv('xgboost_table1.csv')

In [80]:
test_df['position_id'].unique()

array(['Forward', 'Midfielder', 'Defender', 'Goalie'], dtype=object)